### L3

Help a family of 5 people move across the river by boat, and the boat is capable of a maximum of 2 people carrying capacity. Time for travelling of each person in turn is 1s, 3s, 6s, 8s, and 12s. If two people both go on the boat, the boat will travel at the speed of the slower person. Find the minimum time required for the whole family to cross the river. (less than eual to 30s).

In [ ]:
"""
Help a family of 5 people move across the river by boat, and the boat is capable of a maximum of 2 people carrying capacity. Time for travelling of each person in turn is 1s, 3s, 6s, 8s, and 12s. If two people both go on the boat, the boat will travel at the speed of the slower person. Find the minimum time required for the whole family to cross the river. (less than eual to 30s).
"""

from typing import Set
class Entity:
  def __init__(self, name: str, time: int):
    self.name = name
    self.time = time
  
  def __repr__(self):
    return f'{self.name} ({self.time}s)'
  
  def __str__(self):
    return self.__repr__()


class State:
  def __init__(self, left: Set[Entity], right: Set[Entity], time: int):
    self.left = left
    self.right = right
    self.time = time
  
  def __repr__(self):
    return f'time: {self.time}, left: ({self.left}), right: ({self.right})'
  
  def __str__(self):
    return self.__repr__()
  
  def is_final(self):
    return len(self.left) == 0


class Move:
  def __init__(self, isLeft2right: bool, entities: Set[Entity]):
    if not 0 < len(entities) <= 2:
      raise ValueError(f'Invalid number of people crossing the river: Number of people should be 1 or 2, but got {len(entities)}')
    self.isLeft2right = isLeft2right
    self.source = 'left' if isLeft2right else 'right'
    self.destination = 'right' if isLeft2right else 'left'
    self.entities = entities
  
  def __repr__(self):
    return f'{"left" if self.isLeft2right else "right"} -> {"right" if self.isLeft2right else "left"}: {self.entities}'
  
  def __str__(self):
    return self.__repr__()
  
  def validate(self, state: State):
    source = state.left if self.isLeft2right else state.right
    destination = state.right if self.isLeft2right else state.left
    # check if all entities are in source
    entities_not_in_source = self.entities - source
    entities_already_in_destination = self.entities & destination
    if entities_not_in_source:
      raise ValueError(f'Entities {entities_not_in_source} not in {self.source}. Current state: {state}')
    if entities_already_in_destination:
      raise ValueError(f'Entities {entities_already_in_destination} already in {self.destination}. Current state: {state}')
  
  def apply(self, state: State) -> State:
    # check if move is valid
    self.validate(state)
    # apply move
    left = state.left.copy()
    right = state.right.copy()
    if self.isLeft2right:
      left -= self.entities
      right |= self.entities
    else:
      left |= self.entities
      right -= self.entities


In [2]:
class Entity:
  def __init__(self, name: str, time: int):
    self.name = name
    self.time = time
  
  def __repr__(self):
    return f'{self.name} ({self.time}s)'
  
  def __str__(self):
    return self.__repr__()
  
  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return self.name == other.name
  
  def __hash__(self):
    return hash(self.name)

a1 = Entity("Popeye", 12)
a2 = Entity("Popeye", 12)

a1 == a2
set([a1, a2])

{Popeye (12s)}